In [1]:
import pandas as pd
import json
# from utils.preprocessing import generate_pair_data
import os
import numpy as np
from sklearn.metrics import mean_squared_error


In [ ]:
pairs = {eval(k): v for k,v in dict(json.loads(open("data/info/pairs.json", "r").read())).items()}

for pair in pairs.keys():
    df_A = pd.read_pickle("data/raw/" + pair[0] + ".zip")  # needs pickle5 compression (python 3.8)
    df_B = pd.read_pickle("data/raw/" + pair[1] + ".zip")  # needs pickle5 compression (python 3.8)
    pair_df = generate_pair_data(df_A, df_B, "1999-01-01", "2021-03-01")
    pair_df.to_pickle(f"data/processed/{pair[0].upper()}-{pair[1].upper()}.zip")
    print(f"saved data/processed/{pair[0].upper()}-{pair[1].upper()}.zip")

### TODO: FIX BELOW

In [ ]:
model = ElasticNet()
model.fit(np.array(pair_df.loc["2000-01-01":"2014-12-31", :].drop(columns="Return Diff (t+1)")),
          np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"]))
pred_diff = model.predict(np.array(pair_df.loc["2015-01-01":, :].drop(columns="Return Diff (t+1)")))
len(pred_diff)

In [ ]:
len(np.array(pair_df.loc["2015-01-01":, "Return Diff (t+1)"]))

In [ ]:
print(np.std(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.mean(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.min(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.max(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
mean_squared_error(np.array(pair_df.loc["2015-01-01":, "Return Diff (t+1)"]), pred_diff)

In [ ]:
annualized_return(df_A.loc[:,["Simple Return", "Log Return"]])

In [ ]:
from utils.models import GeometricBrownianMotion

GeometricBrownianMotion(1, 0.03, 0.05).simulate()

### Other Models

In [13]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

def make_model(model, param_value):
    # acceptable model
    models = ['svr', 'forest', 'adaboost', 'gradboost', 'nn']
    assert model in models
    
    if model == 'svr': return SVR(C=param_value)
    elif model == 'forest': return RandomForestRegressor(n_estimators=param_value)
    elif model == 'adaboost': return AdaBoostRegressor(n_estimators=param_value)
    elif model == 'gradboost': return GradientBoostingRegressor(n_estimators=param_value)
    # hidden_layer_sizes is a tuple, where the ith element is the size of the ith hidden layer
    # nn = neural network
    elif model == 'nn': return MLPRegressor(hidden_layer_sizes=param_value) 
    else: raise Exception('unacceptable model type')
    
    
modelparams = {
    'svr': [10**x for x in range(-3,4)],
    'forest': range(50,100,10),
    'adaboost': range(50,100,10),
    'gradboost': range(50,100,10),
    'nn': [(x,y) for x in range(50,100,10) for y in range(50,100,10)]
}


In [14]:
data_path = r'data//processed'
data_filenames = os.listdir(data_path)

for data_file in data_filenames:
    pairs = data_file[:-4].split('-')
    print("Pairs Evaluated:", pairs[0], pairs[1])
    
    pair_df = pd.read_pickle(os.path.join(data_path, data_file))
    X = np.array(pair_df.loc["2000-01-01":"2014-12-31", :].drop(columns="Return Diff (t+1)"))
    y = np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])
    X_test = np.array(pair_df.loc["2015-01-01":, :].drop(columns="Return Diff (t+1)"))
    y_test = np.array(pair_df.loc["2015-01-01":, "Return Diff (t+1)"])
    
    for key in modelparams:
        print("Model:", key)
        for param in modelparams[key]:
            print("Parameter Value:", param)
            model = make_model(key, param)
            model.fit(X,y)

            pred = model.predict(X_test)
            mse = mean_squared_error(y_test, pred)
            print("MSE:", mse)
            print()
        print("======================\n")
    
    

Pairs Evaluated: ADX TY
Model: svc
Parameter Value: 0.001
MSE: 0.0004730368663426111

Parameter Value: 0.01
MSE: 0.0004730368663426111

Parameter Value: 0.1
MSE: 0.0004730368663426111

Parameter Value: 1
MSE: 0.0004730368663426111

Parameter Value: 10
MSE: 0.0004730368663426111

Parameter Value: 100
MSE: 0.0004730368663426111

Parameter Value: 1000
MSE: 0.0004730368663426111


Model: forest
Parameter Value: 50
MSE: 7.656879431416973e-07

Parameter Value: 60
MSE: 7.833038938049414e-07

Parameter Value: 70
MSE: 7.492476800887097e-07

Parameter Value: 80
MSE: 8.004149685475058e-07

Parameter Value: 90
MSE: 7.759088896429089e-07


Model: adaboost
Parameter Value: 50
MSE: 1.5951873545652025e-06

Parameter Value: 60
MSE: 1.5673122081207518e-06

Parameter Value: 70
MSE: 1.7688956185551402e-06

Parameter Value: 80
MSE: 1.3749028804352655e-06

Parameter Value: 90
MSE: 2.049402068378452e-06


Model: gradboost
Parameter Value: 50
MSE: 6.319882558430266e-07

Parameter Value: 60
MSE: 5.962695154464

MSE: 0.00027590217147803464

Parameter Value: 60
MSE: 0.00027369286674580433

Parameter Value: 70
MSE: 0.00027287988066297485

Parameter Value: 80
MSE: 0.000268532606612306

Parameter Value: 90
MSE: 0.00026906899559910784


Model: adaboost
Parameter Value: 50
MSE: 0.0002779811534081937

Parameter Value: 60
MSE: 0.000276492555621025

Parameter Value: 70
MSE: 0.0002656570572726512

Parameter Value: 80
MSE: 0.0002762836771814894

Parameter Value: 90
MSE: 0.0002776758795296497


Model: gradboost
Parameter Value: 50
MSE: 0.00026625697161301415

Parameter Value: 60
MSE: 0.00026422951038000247

Parameter Value: 70
MSE: 0.00026193678128848263

Parameter Value: 80
MSE: 0.00027256688765990746

Parameter Value: 90
MSE: 0.0002605362674678712


Model: nn
Parameter Value: (50, 50)
MSE: 0.001316500421935598

Parameter Value: (50, 60)
MSE: 0.0008431566902750502

Parameter Value: (50, 70)
MSE: 0.0007576011617921335

Parameter Value: (50, 80)
MSE: 0.0011160783699612883

Parameter Value: (50, 90)
MSE: 0.

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').